In [ ]:
import csv, json, sys, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
models = False
if models = True:
    !pip install 'tensorflow==1.15.0'
    import tensorflow as tf
    assert tf.__version__=='1.15.0'
    !apt-get update
    !apt-get install -y cmake libopenmpi-dev python3-dev zlib1g-dev
    !pip install "stable-baselines[mpi]==2.9.0"

    from stable_baselines import PPO1

## Intro

Format of the dataset: 
> {"board": [0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 2, 1, 0, 2, 0, 0, 0, 1, 1, 1, 2, 1, 0, 1, 2], "score": -2, "move score": [-3, -4, -4, -2, -6, -5, -4]}

Each row is a json dictionary with the following fields:
* "board": kaggle format of a connect4 baord, 
* "score": Score for the position, 
* "move score": Array of 7 scores corresponding to a play in each of the 7 columns

A note on the scores in the dataset:
* Score = 0: Game will be a draw
* Score > 0: Current player will win (the bigger the number the sooner the player will win). The score is the half the ammount of plies from the end the game will be won. So +5 is means the win will be in ply 42 - 2*5 = 32. 
* Score < 0: Current player will lose (the bigger the number the sooner the player will lose)
* Score = -99: simply indicates that that was not a legal move.


### Let's analyze the built in agents

In [ ]:
import json
!pip install kaggle_environments
import kaggle_environments
from kaggle_environments.utils import structify

# requires "/content/refmoves1k_kaggle.csv" aka "good moves archive"

In [ ]:

def win_loss_draw(score):
    if score>0: 
        return 'win'
    if score<0: 
        return 'loss'
    return 'draw'

def score(agent, max_lines = 1000):
    #Scores a connect-x agent with the dataset
    print("scoring ",agent)
    count = 0
    good_move_count = 0
    perfect_move_count = 0
    observation = structify({'mark': None, 'board': None})
    #with open("/kaggle/input/1k-connect4-validation-set/refmoves1k_kaggle") as f:
    with open("/content/refmoves1k_kaggle.csv") as f:
        for line in f:
            count += 1
            data = json.loads(line)
            observation.board = data["board"]
            # find out how many moves are played to set the correct mark.
            ply = len([x for x in data["board"] if x>0])
            if ply&1:
                observation.mark = 2
            else:
                observation.mark = 1
            
            #call the agent
            agent_move = agent(observation,env.configuration)
            
            moves = data["move score"]
            perfect_score = max(moves)
            perfect_moves = [ i for i in range(7) if moves[i]==perfect_score]

            if(agent_move in perfect_moves):
                perfect_move_count += 1

            if win_loss_draw(moves[agent_move]) == win_loss_draw(perfect_score):
                good_move_count += 1

            if count == max_lines:
                break

        print("perfect move percentage: ",perfect_move_count/count)
        print("good moves percentage: ",good_move_count/count)


###Output should be:

> scoring **random_agent**  
> perfect move percentage:  0.22  
> good moves percentage:  0.67  
---
> scoring **negamax_agent**  
> perfect move percentage:  0.4  
> good moves percentage:  0.71  
___
Some more references:  
A neural net that I use in my best agent (1267 score on 2/24/20) score as follows:
> perfect move percentage:  0.737  
> good moves percentage:  0.939


## json

In [ ]:
fileInput = '/content/5471908.json'
inputFile = open(fileInput) #open json file
data = json.load(inputFile) #load json content
#json_df = pd.read_json(inputFile)
inputFile.close() #close the input file

In [ ]:
#fileOutput = '/content/5471908.csv'
#outputFile = open(fileOutput, 'w') #load csv file
#output = csv.writer(outputFile) #create a csv.write
#output.writerow(data[0].keys())  # header row
#for row in data:
#    output.writerow(row.values()) #values row

In [ ]:
datalist = list(data.values())
for key, item in zip(data.keys(), datalist):
    print(key,":",item)

In [ ]:
steps = data['steps']
steps[len(steps) - 1]

In [ ]:
s = steps[len(steps) - 1]
s[0]['observation']['board']

In [ ]:
player_actions = []
opp_actions =  []
board_list = []
for s in steps:
    player_actions += [ s[0]['action'] ]
    opp_actions += [ s[1]['action'] ]
    board_list += [ s[0]['observation']['board'] ]
#player_actions, opp_actions, 
board_list

In [ ]:
df = pd.DataFrame(board_list)
df["play"] = player_actions
df["opp"] = opp_actions
df

In [ ]:
board -= 1
print ("Board:",board)
print ("Player played:", df["play"][board])
print ("Opponent played:", df["opp"][board])
grid = np.array(df.iloc[board])#
plt.imshow(grid[:-2].reshape((6,7)))

##csv

###pandas

In [ ]:
good_moves_df = pd.read_csv('/content/good_move_archive.csv', sep=';', header=None)
good_moves_df = good_moves_df.drop(1,axis=1)
good_moves_series = good_moves_df[0]
good_moves_series

In [ ]:
good_moves_series.to_csv('/content/good_moves.csv', header=False)

In [ ]:
#del good_moves_df
good_moves_df = pd.read_csv('/content/good_moves.csv', index_col=0, header=None)
good_moves_df

In [ ]:
gm_arr = good_moves_df.values
gm = gm_arr[1]
gm

In [ ]:
gm_str = gm[0]
gm_str.split(sep='')
for x in gm_str:
    print(x)

###semi

In [ ]:
gmsemi_df = pd.read_csv('/content/good_move_semic.csv', sep=';', header=None)
gmsemi_df[0]

In [ ]:
boards = [b.split(": ")[1] for b in gmsemi_df[0]]
boards = [b.replace("[","").replace("]","") for b in boards]
boards = [b_str.split(", ") for b_str in boards]
for i, blist in enumerate(boards):
    boards[i] = [int(b) for b in blist]
boards = np.array(boards)
boards

In [ ]:
listlist = [list(b) for b in boards]
listlist

In [ ]:
moves = [m.split(": ")[1] for m in gmsemi_df[2]]
moves = [m.replace("[","").replace("]","") for m in moves]
moves = [m_str.split(", ") for m_str in moves]
for i, mlist in enumerate(moves):
    moves[i] = [int(m) for m in mlist]
moves = np.array(moves)
moves

In [ ]:
scores = [s.split(": ")[1] for s in gmsemi_df[1]]
scores = [s.replace("[","").replace("]","") for s in scores]
scores = [s.split(", ") for s in scores]
for i, slist in enumerate(scores):
    scores[i] = [int(s) for s in slist]
scores = np.array(scores)
scores = scores.flatten().tolist()
scores

In [ ]:
pd.DataFrame(moves).to_csv("/content/moves.csv")

In [ ]:
import os
with open() as fh:    
    moves.tofile(fh)

In [ ]:
scores[11],list(moves[11]),list(boards[11])

In [ ]:
labels = [np.argmax(move) for move in moves] 
np.array(labels)

In [ ]:
training_set = pd.DataFrame(listlist)
#training_set["boards"] = boards
training_set["labels"] = labels
training_set.to_csv('training_set')

In [ ]:
del training_set
training_set = pd.read_csv('/content/training_set', index_col=0)
training_set